In [ ]:
from dotenv import load_dotenv

from langgraph.graph import END, StateGraph

from graph.consts import RETRIEVE, GRADE_DOCUMENTS, GENERATE, WEBSEARCH
from graph.nodes import generate, grade_documents, retrieve, web_search
from graph.state import GraphState

load_dotenv()


# define conditional edge as a function that outputs the selected path
def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")

    if state["web_search"]:
        print(
            "---DECISION: NOT ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return WEBSEARCH
    else:
        print("---DECISION: GENERATE---")
        return GENERATE


# build the graph based on the nodes that you already defined.
# here we connect the nodes
# the constructor of our graph is GraphState instead of MessageGraph, bcs you defined a specific state as the memory of graph.
workflow = StateGraph(GraphState)

workflow.add_node(RETRIEVE, retrieve)
workflow.add_node(GRADE_DOCUMENTS, grade_documents)
workflow.add_node(GENERATE, generate)
workflow.add_node(WEBSEARCH, web_search)

# add edges and conditional edges
workflow.set_entry_point(RETRIEVE)
workflow.add_edge(RETRIEVE, GRADE_DOCUMENTS)
workflow.add_conditional_edges(
    GRADE_DOCUMENTS,
    decide_to_generate,
    # to have more control, you can add mapping routes
    {
        WEBSEARCH: WEBSEARCH,
        GENERATE: GENERATE,
    },
)
workflow.add_edge(WEBSEARCH, GENERATE)
workflow.add_edge(GENERATE, END)

app = workflow.compile()

app.get_graph().draw_mermaid_png(output_file_path="graph.png")

## Q. when do you need conditional edge in your graph?
when you have more than one route and you want to decide where to go in the graph based on your results and metrics.

## Q. explain conditional edge above?

```python
workflow.add_conditional_edges(
    GRADE_DOCUMENTS,           # Source node
    decide_to_generate,        # Decision function
    {                          # Mapping dictionary
        WEBSEARCH: WEBSEARCH,
        GENERATE: GENERATE,
    },
)
```

`Source Node (GRADE_DOCUMENTS)`: The node from which the conditional routing happens<br>
`Decision Function (decide_to_generate)`: The function that determines which path to take <br>
`Mapping Dictionary`: Maps the function's return values to actual destination nodes

### When is the Mapping Dictionary Useful in LangGraph Conditional Edges?
The mapping dictionary in conditional edges becomes useful when you want different routing logic than simple direct mapping. Here are the key scenarios:
1. Multiple Conditions Leading to Same Node
python
```def decide_next_step(state):
    if state["error_count"] > 3:
        return "CRITICAL_ERROR"
    elif state["validation_failed"]:
        return "VALIDATION_ERROR"
    elif state["timeout"]:
        return "TIMEOUT_ERROR"
    else:
        return "SUCCESS"

workflow.add_conditional_edges(
    "PROCESS_DATA",
    decide_next_step,
    {
        "CRITICAL_ERROR": "ERROR_HANDLER",    # Different conditions
        "VALIDATION_ERROR": "ERROR_HANDLER",  # Same destination
        "TIMEOUT_ERROR": "ERROR_HANDLER",     # Same destination
        "SUCCESS": "GENERATE_REPORT",
    }
)```

2. Function Returns Don't Match Node Names
python

```def choose_model(state):
    if state["complexity"] == "high":
        return "use_advanced_model"  # Function return
    else:
        return "use_basic_model"     # Function return

workflow.add_conditional_edges(
    "ANALYZE_QUERY",
    choose_model,
    {
        "use_advanced_model": "GPT4_NODE",     # Actual node name
        "use_basic_model": "GPT3_NODE",        # Actual node name
    }
)
```

3. Complex Multi-Path Routing
python
```def route_by_user_type(state):
    user_role = state["user"]["role"]
    urgency = state["request"]["urgency"]
    
    if user_role == "admin":
        return "admin_path"
    elif user_role == "premium" and urgency == "high":
        return "priority_queue"
    elif user_role == "premium":
        return "premium_queue"
    else:
        return "standard_queue"

workflow.add_conditional_edges(
    "CLASSIFY_REQUEST",
    route_by_user_type,
    {
        "admin_path": "ADMIN_PROCESSOR",
        "priority_queue": "PRIORITY_HANDLER", 
        "premium_queue": "PREMIUM_HANDLER",
        "standard_queue": "STANDARD_HANDLER",
    }
)```

4. When You DON'T Need the Mapping Dictionary
If your function returns match your node names exactly, you can omit it:

python
```def simple_decision(state):
    return "NODE_A" if state["condition"] else "NODE_B"

# This works without mapping since return values match node names
workflow.add_conditional_edges("SOURCE", simple_decision)```